In [11]:
import boto3
import pandas as pd
import numpy as np
import configparser
import psycopg2

# Config setup

In [7]:
config = configparser.ConfigParser()
config.read('config.cfg')

['config.cfg']

# AWS client connection

In [8]:
access_key = config.get('IAM', 'ACCESS_KEY')
secret_key = config.get('IAM', 'SECRET_KEY')
region = config.get('REGION', 'REGION_NAME')

aws_rds_conn = boto3.client('rds', 
                            aws_access_key_id=access_key, 
                            aws_secret_access_key=secret_key, 
                            region_name=region)

# AWS RDS creation

In [9]:
db_instance_id = config.get('DB', 'DB_INSTANCE_ID')
db_name = config.get('DB', 'DB_NAME')
db_username = config.get('DB', 'DB_USERNAME')
db_password = config.get('DB', 'DB_PASSWORD')
db_port = config.get('DB', 'DB_PORT')

vpc_security_group = config.get('VPC', 'SECURITY_GROUP')

try:
    response = aws_rds_conn.create_db_instance(
        DBInstanceIdentifier=db_instance_id,
        DBName=db_name,
        MasterUsername=db_username,
        MasterUserPassword=db_password,
        Port=int(db_port),
        DBInstanceClass='db.t3.micro',
        Engine='postgres',
        PubliclyAccessible=True,
        AllocatedStorage=20,
        VpcSecurityGroupIds=[vpc_security_group]
    )
    print(response)
except aws_rds_conn.exceptions.DBInstanceAlreadyExistsFault:
    print('La instancia ya existe')
except Exception as ex:
    print(f"Error: ${ex}")

{'DBInstance': {'DBInstanceIdentifier': 'shop', 'DBInstanceClass': 'db.t3.micro', 'Engine': 'postgres', 'DBInstanceStatus': 'creating', 'MasterUsername': 'admin_postgress', 'DBName': 'shop', 'AllocatedStorage': 20, 'PreferredBackupWindow': '06:19-06:49', 'BackupRetentionPeriod': 1, 'DBSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-475da528', 'Status': 'active'}], 'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres16', 'ParameterApplyStatus': 'in-sync'}], 'DBSubnetGroup': {'DBSubnetGroupName': 'default', 'DBSubnetGroupDescription': 'default', 'VpcId': 'vpc-a0b0bdc8', 'SubnetGroupStatus': 'Complete', 'Subnets': [{'SubnetIdentifier': 'subnet-cb70ba87', 'SubnetAvailabilityZone': {'Name': 'us-east-2c'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-436b0939', 'SubnetAvailabilityZone': {'Name': 'us-east-2b'}, 'SubnetOutpost': {}, 'SubnetStatus': 'Active'}, {'SubnetIdentifier': 'subnet-7e4c7816', 'SubnetAvailabilityZone': {'Na

# RDS host name

In [12]:
try:
    instance = aws_rds_conn.describe_db_instances(DBInstanceIdentifier=db_instance_id)
    RDS_HOST_NAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOST_NAME)
except Exception as ex:
    print('Error: ', ex)

shop.cnkf6pwg5si3.us-east-2.rds.amazonaws.com


# Creation of transactional system

In [17]:
import create_transactional

try:
    db_pg_conn = psycopg2.connect(
        database=db_name,
        user=db_username,
        password=db_password,
        port=db_port,
        host=RDS_HOST_NAME
    )
    cursor = db_pg_conn.cursor()
    cursor.execute(create_transactional.DDL_QUERY)
    db_pg_conn.commit()
except Exception as ex:
    print('Error: ', ex)

# Faker helper functions

In [41]:
from faker import Faker
import random

fake = Faker()

# Generar datos para "categoria"
categorias_con_descripciones = {
    'Electrónica': 'Dispositivos y gadgets tecnológicos de última generación.',
    'Ropa Hombres': 'Moda y accesorios para el vestuario masculino.',
    'Ropa Mujeres': 'Colecciones y tendencias de moda femenina.',
    'Juguetes': 'Juguetes educativos y de entretenimiento para niños de todas las edades.',
    'Alimentos': 'Productos alimenticios frescos y procesados para tu despensa.',
    'Bebidas': 'Variedad de bebidas refrescantes, alcohólicas y no alcohólicas.',
    'Libros': 'Literatura, ficción, educativos y más géneros para todos los lectores.',
    'Artículos de Oficina': 'Suministros y mobiliario para mejorar tu espacio de trabajo.',
    'Deportes': 'Equipamiento y ropa deportiva para todas tus actividades físicas.',
    'Zapatos': 'Calzado cómodo y estilizado para hombres, mujeres y niños.',
    'Accesorios': 'Complementos perfectos para realzar cualquier atuendo.',
    'Belleza y Cuidado Personal': 'Productos para cuidado de la piel, cabello y belleza general.',
    'Hogar y Cocina': 'Artículos para equipar y decorar tu hogar y cocina.',
    'Jardinería': 'Herramientas y decoraciones para mantener tu jardín hermoso.',
    'Mascotas': 'Alimentos, juguetes y accesorios para el bienestar de tus mascotas.',
    'Automóviles': 'Accesorios, herramientas y tecnología para tu coche.',
    'Música': 'Instrumentos musicales y accesorios para aficionados y profesionales.',
    'Videojuegos': 'Consolas, videojuegos y accesorios para gamers.',
    'Fotografía': 'Cámaras, lentes y accesorios para fotógrafos.',
    'Celulares y Accesorios': 'Los últimos smartphones y complementos para estar siempre conectado.',
    'Computación': 'Hardware, software y periféricos para tus necesidades informáticas.',
    'Audio y Video': 'Equipos de sonido y video para una experiencia multimedia completa.',
    'Salud y Bienestar': 'Productos para mantener un estilo de vida saludable y activo.',
    'Joyería': 'Joyas finas y relojería para ocasiones especiales.',
    'Relojes': 'Una amplia selección de relojes para cada estilo y ocasión.'
}

def generar_categorias():
    return [
        {
            'nombre': nombre, 
            'descripcion': descripcion,
            'estado': True
        } for nombre, descripcion in categorias_con_descripciones.items()
    ]

# Generar datos para "rol"
roles_tiendas = [
    ('Administrador', 'Responsable de gestionar la tienda, incluidos productos, personal y finanzas'),
    ('Vendedor', 'Encargado de las ventas y la atención al cliente en el piso de ventas'),
    ('Cajero', 'Gestiona las transacciones de pago y emite recibos a los clientes'),
    ('Gerente de Producto', 'Supervisa la selección, precios y promoción de los productos'),
    ('Soporte Técnico', 'Proporciona asistencia técnica para productos electrónicos y software'),
    ('Gerente de Marketing', 'Desarrolla estrategias de marketing y publicidad para la tienda'),
    ('Gerente de RR.HH.', 'Administra la contratación, formación y políticas de recursos humanos'),
]
def generar_roles():
    return [
        {
            'nombre': rol[0], 
            'descripcion': rol[1],
            'estado': True
        } for rol in roles_tiendas
    ]


# Generar datos para "articulo"
def generar_articulo(idcategoria):
    return {
        'idcategoria': idcategoria,
        'codigo': fake.bothify(text='???-###'),
        'nombre': fake.word(),
        'precio_venta': round(random.uniform(10, 1000), 2),
        'stock': fake.random_int(min=0, max=100),
        'descripcion': fake.text(max_nb_chars=50),
        'imagen': fake.file_name(category='image'),
        'estado': fake.boolean()
    }

# Generar datos para "persona"
def generar_persona():
    tipo_persona = random.choice(['cliente', 'proveedor'])
    tipo_documento = random.choice(['DNI', 'CE', 'PAS'])
    return {
        'tipo_persona': tipo_persona,
        'nombre': fake.name(),
        'tipo_documento': tipo_documento,
        'num_documento': fake.bothify(text='########'),
        'direccion': fake.address(),
        'telefono': fake.phone_number(),
        'email': fake.email()
    }


# Generar datos para "usuario"
def generar_usuario(idrol):
    tipo_documento = random.choice(['DNI', 'CE', 'PAS'])
    return {
        'idrol': idrol,
        'nombre': fake.name(),
        'tipo_documento': tipo_documento,
        'num_documento': fake.bothify(text='########'),
        'direccion': fake.address(),
        'telefono': fake.phone_number(),
        'email': fake.email(),
        'clave': fake.binary(length=16),
        'estado': fake.boolean()
    }

# Generar datos para "venta"
def generar_venta(idcliente, idusuario):
    tipo_comprobante = random.choice(['Factura', 'Boleta'])
    return {
        'idcliente': idcliente,
        'idusuario': idusuario,
        'tipo_comprobante': tipo_comprobante,
        'serie_comprobante': fake.bothify(text='???'),
        'num_comprobante': fake.bothify(text='#######'),
        'fecha': fake.date_time_this_year(),
        'impuesto': round(random.uniform(0, 30), 2),
        'total': round(random.uniform(100, 1000), 2),
        'estado': random.choice(['Pagado', 'Pendiente', 'Anulado'])
    }

# Generar datos para "detalle_venta"
def generar_detalle_venta(idventa, idarticulo):
    return {
        'idventa': idventa,
        'idarticulo': idarticulo,
        'cantidad': fake.random_int(min=1, max=10),
        'precio': round(random.uniform(10, 100), 2),
        'descuento': round(random.uniform(0, 10), 2)
    }

# Generar datos para "ingreso"
def generar_ingreso(idproveedor, idusuario):
    tipo_comprobante = random.choice(['Factura', 'Boleta', 'Ticket'])
    return {
        'idproveedor': idproveedor,
        'idusuario': idusuario,
        'tipo_comprobante': tipo_comprobante,
        'serie_comprobante': fake.bothify(text='???'),
        'num_comprobante': fake.bothify(text='#######'),
        'fecha': fake.date_time_this_year(),
        'impuesto': round(random.uniform(0, 30), 2),
        'total': round(random.uniform(100, 1000), 2),
        'estado': random.choice(['Recibido', 'Pendiente', 'Anulado'])
    }

# Generar datos para "detalle_ingreso"
def generar_detalle_ingreso(idingreso, idarticulo):
    return {
        'idingreso': idingreso,
        'idarticulo': idarticulo,
        'cantidad': fake.random_int(min=1, max=100),
        'precio': round(random.uniform(10, 100), 2)
    }

# Insertar datos
def insertData2SQL(data_dict, table_name, driver):
    df_data = pd.DataFrame.from_records(data_dict)
    try:
        response = df_data.to_sql(table_name, driver, index=False, if_exists='append')
        print(f"Se han insertado {response} nuevos registros")
    except Exception as ex:
        print('Error: ', ex)


# Cargando datos

In [42]:
driver = f"""postgresql://{db_username}:{db_password}@{RDS_HOST_NAME}:{db_port}/{db_name}"""

In [43]:
# categorias
insertData2SQL(generar_categorias(), 'categoria', driver)

Se han insertado 25 nuevos registros


In [44]:
# roles
insertData2SQL(generar_roles(), 'rol', driver)

Se han insertado 7 nuevos registros
